In [3]:
import cv2
import mediapipe as mp
import numpy as np 
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [ ]:
# VIDEO FEED
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow('mediapipe feed', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

# 1. Make Detections

In [4]:
cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )
        # mp_drawing ke 1 untuk mengatur titik dan mp_drawing ke 2 untuk garis
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# 2, Determining joints

<img src="img_pose.png" style="height: 500px;"/>

In [4]:
cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landarks
        try:
            landmarks = results.pose_landmarks.landmark
        except:
            pass
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )
        # mp_drawing ke 1 untuk mengatur titik dan mp_drawing ke 2 untuk garis
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [5]:
len(landmarks)

33

In [6]:
landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]

x: 0.7318251
y: 0.9456895
z: -0.43386635
visibility: 0.99267423

In [7]:
mp_pose.PoseLandmark.LEFT_SHOULDER.value

11

# 3. Calculate Angles

In [14]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360-angle
    
    return angle

Fungsi ini menghitung sudut antara tiga titik menggunakan trigonometri. Sudut diukur dalam derajat dan dihitung sebagai perbedaan antara dua arah sudut menggunakan fungsi arctangent.

In [12]:
shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

In [13]:
shoulder, elbow, wrist

([0.7318251132965088, 0.9456894993782043],
 [0.8412498235702515, 1.3371334075927734],
 [0.8448547124862671, 1.7114145755767822])

In [15]:
calculate_angle(shoulder, elbow, wrist)

164.93395845091243

In [20]:
cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get Coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            # Calculate angle
            angle = calculate_angle(shoulder, elbow, wrist)
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                        tuple(np.multiply(elbow, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            #print(landmarks)
        except:
            pass
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )
        # mp_drawing ke 1 untuk mengatur titik dan mp_drawing ke 2 untuk garis
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

[x: 0.49194068
y: 0.68327415
z: -0.77453566
visibility: 0.99898463
, x: 0.5073165
y: 0.64342296
z: -0.7328656
visibility: 0.9973838
, x: 0.5182666
y: 0.64365935
z: -0.733031
visibility: 0.99775726
, x: 0.5286442
y: 0.64464897
z: -0.7331557
visibility: 0.99705386
, x: 0.4720785
y: 0.6467871
z: -0.7339294
visibility: 0.9977816
, x: 0.46115908
y: 0.6489507
z: -0.7336382
visibility: 0.99827456
, x: 0.4515438
y: 0.65139836
z: -0.7341389
visibility: 0.9981299
, x: 0.54822195
y: 0.6667373
z: -0.44901046
visibility: 0.9967443
, x: 0.44131365
y: 0.6702568
z: -0.44509324
visibility: 0.99866366
, x: 0.5189273
y: 0.72215676
z: -0.66765475
visibility: 0.9987979
, x: 0.47349453
y: 0.725017
z: -0.66635543
visibility: 0.9992974
, x: 0.66170037
y: 0.90306026
z: -0.2464566
visibility: 0.99098855
, x: 0.3502344
y: 0.9124419
z: -0.29483837
visibility: 0.9950895
, x: 0.7411105
y: 1.1545924
z: -0.22481997
visibility: 0.12017573
, x: 0.24579039
y: 1.1843976
z: -0.30119252
visibility: 0.28899518
, x: 0.776251